In [1]:
!git clone https://github.com/AleksL04/ML-Project.git
%cd ML-Project

Cloning into 'ML-Project'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 137 (delta 65), reused 120 (delta 48), pack-reused 0 (from 0)
Receiving objects: 100% (137/137), 53.06 KiB | 4.42 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/ML-Project


In [2]:
!git pull

Already up to date.


In [3]:
%%capture
!pip install --upgrade gensim
!gdown --folder 1rl_TJMHtcP-S0fTHNUpr9oi8Ndz2Y5z4 -O project_data
!gdown --folder 1RmC_We2lfsJontwrxlc4dzqEY_mNy-UW -O word2vec_model

!pip install --upgrade scikeras
!pip install "scikit-learn<1.6"


In [4]:
ls -l

total 36
-rw-r--r-- 1 root root 16782 Dec 12 01:22 main_console.ipynb
drwxr-xr-x 2 root root  4096 Dec 12 01:22 model_code/
-rw-r--r-- 1 root root  1550 Dec 12 01:22 process_df.py
drwxr-xr-x 2 root root  4096 Dec 12 01:23 project_data/
drwxr-xr-x 2 root root  4096 Dec 12 01:23 word2vec_model/


In [5]:
import pandas as pd
import numpy as np

In [6]:
train_df = pd.read_csv('project_data/train.csv')
valid_df = pd.read_csv('project_data/valid.csv')
test_df = pd.read_csv('project_data/test.csv')

In [7]:
from process_df import process_df

X_train = process_df(train_df)
y_train = train_df['label'].values

X_valid = process_df(valid_df)
y_valid = valid_df['label'].values

X_test = process_df(test_df)
y_test = test_df['label'].values

In [8]:
from model_code.LSTM_code import trainLSTM, train_CV_LSTM
from model_code.CNN_code import trainCNN, train_CV_CNN
from model_code.Blender_code import trainBlender
from model_code.Bayes_code import trainBayes, predictBayes

In [9]:
# train_CV_LSTM(X_train, y_train, X_valid, y_valid)
#train_CV_CNN(X_train, y_train, X_valid, y_valid)

In [10]:
LSTM_model = trainLSTM(X_train, y_train, X_valid, y_valid)

test_loss, test_acc = LSTM_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8926 - loss: 0.2498
Test Accuracy: 0.8861


In [11]:
CNN_model = trainCNN(X_train, y_train, X_valid, y_valid)

test_loss, test_acc = CNN_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.8444 - loss: 0.3594
Test Accuracy: 0.8458


In [12]:
Bayes_model, vectorizer = trainBayes(train_df, valid_df)

Validation Accuracy: 0.7541899441340782
Validation Loss: 0.5015656617689024


In [13]:
val_pred_LSTM = LSTM_model.predict(X_valid).flatten()
val_pred_CNN = CNN_model.predict(X_valid).flatten()
_,_,val_pred_Bayes = predictBayes(Bayes_model, valid_df, vectorizer)

X_meta_train = np.column_stack([
    val_pred_LSTM,
    val_pred_CNN,
    val_pred_Bayes
])

y_meta_train = y_valid

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step


In [14]:
# val_pred_LSTM = LSTM_model.predict(X_valid)
# val_pred_CNN = CNN_model.predict(X_valid)
# val_pred_Bayes = predictBayes(Bayes_model, valid_df, vectorizer)
# print(val_pred_Bayes)
# print(val_pred_LSTM)
# X_meta_train = np.column_stack((val_pred_LSTM, val_pred_CNN))

# y_meta_train = y_valid

In [15]:
Blender_model = trainBlender(X_meta_train, y_meta_train)

Epoch 1/100


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.6258 - loss: 0.6499 - learning_rate: 0.0010
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8567 - loss: 0.6322 - learning_rate: 0.0010
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8894 - loss: 0.6113 - learning_rate: 0.0010
Epoch 4/100


/usr/local/lib/python3.12/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
/usr/local/lib/python3.12/dist-packages/keras/src/callbacks/callback_list.py:145: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss,learning_rate.
  callback.on_epoch_end(epoch, logs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8621 - loss: 0.6028 - learning_rate: 0.0010
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8991 - loss: 0.5830 - learning_rate: 0.0010
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8891 - loss: 0.5771 - learning_rate: 0.0010
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8811 - loss: 0.5645 - learning_rate: 0.0010
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8860 - loss: 0.5500 - learning_rate: 0.0010
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8806 - loss: 0.5449 - learning_rate: 0.0010
Epoch 10/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8946 - loss: 0.5241 - learning_rate: 0.0010
Epoch 11/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8933 - loss: 0.5146 - learning_rate: 0.0010
Epoch 12/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8877 - loss: 0.5051 - learning_rate: 0.0010
Epoch 13/100
23/23 ━━━━━━━━━

In [18]:
test_pred_LSTM = LSTM_model.predict(X_test).flatten()
test_pred_CNN = CNN_model.predict(X_test).flatten()
_,_,test_pred_Bayes = predictBayes(Bayes_model, test_df, vectorizer)

X_meta_test = np.column_stack([
    test_pred_LSTM,
    test_pred_CNN,
    test_pred_Bayes
])


# test_pred_LSTM = LSTM_model.predict(X_test)
# test_pred_CNN = CNN_model.predict(X_test)

# X_meta_test = np.column_stack((test_pred_LSTM, test_pred_CNN))

test_loss, test_acc = Blender_model.evaluate(X_meta_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8970 - loss: 0.2734
Test Accuracy: 0.8965
